# REINFORCE in TensorFlow

This notebook implements a basic reinforce algorithm a.k.a. policy gradient for CartPole env.

It has been deliberately written to be as simple and human-readable.


The notebook assumes that you have [openai gym](https://github.com/openai/gym) installed.

In case you're running on a server, [use xvfb](https://github.com/openai/gym#rendering-on-a-server)

In [1]:
import gym
import numpy as np, pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

env = gym.make("CartPole-v0")

#gym compatibility: unwrap TimeLimit
if hasattr(env,'env'):
    env=env.env

env.reset()
n_actions = env.action_space.n
state_dim = env.observation_space.shape



# Building the policy network

For REINFORCE algorithm, we'll need a model that predicts action probabilities given states.

For numerical stability, please __do not include the softmax layer into your network architecture__. 

We'll use softmax or log-softmax where appropriate.

In [2]:
import tensorflow as tf

#create input variables. We only need <s,a,R> for REINFORCE
states = tf.placeholder('float32',(None,)+state_dim,name="states")
actions = tf.placeholder('int32',name="action_ids")
cumulative_rewards = tf.placeholder('float32', name="cumulative_returns")

/home/informix/.local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:526: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/informix/.local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:527: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/informix/.local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:528: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/informix/.local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.

In [3]:
import keras
import keras.layers as L

nn = keras.models.Sequential()
nn.add(L.Dense(32, activation='relu', input_shape=state_dim))
nn.add(L.Dense(32, activation='relu'))
nn.add(L.Dense(n_actions, activation='linear'))


logits = nn(states)

policy = tf.nn.softmax(logits)
log_policy = tf.nn.log_softmax(logits)

Instructions for updating:
Colocations handled automatically by placer.


Using TensorFlow backend.


In [4]:
#utility function to pick action in one given state
get_action_proba = lambda s: policy.eval({states:[s]})[0] 

#### Loss function and updates

We now need to define objective and update over policy gradient.

Our objective function is

$$ J \approx  { 1 \over N } \sum  _{s_i,a_i} \pi_\theta (a_i | s_i) \cdot G(s_i,a_i) $$


Following the REINFORCE algorithm, we can define our objective as follows: 

$$ \hat J \approx { 1 \over N } \sum  _{s_i,a_i} log \pi_\theta (a_i | s_i) \cdot G(s_i,a_i) $$

When you compute gradient of that function over network weights $ \theta $, it will become exactly the policy gradient.


In [5]:
#get probabilities for parti
indices = tf.stack([tf.range(tf.shape(log_policy)[0]),actions],axis=-1)
log_policy_for_actions = tf.gather_nd(log_policy,indices)

In [6]:
# policy objective as in the last formula. please use mean, not sum.
# note: you need to use log_policy_for_actions to get log probabilities for actions taken

J = tf.reduce_mean(log_policy_for_actions * cumulative_rewards)

In [7]:
#regularize with entropy
entropy = tf.reduce_sum(policy * log_policy, 1)

In [8]:
#all network weights
all_weights = tf.get_collection(tf.GraphKeys.TRAINABLE_VARIABLES)

#weight updates. maximizing J is same as minimizing -J. Adding negative entropy.
loss = -J -0.1 * entropy

update = tf.train.AdamOptimizer().minimize(loss,var_list=all_weights)

Instructions for updating:
Use tf.cast instead.


### Computing cumulative rewards

In [9]:
def get_cumulative_rewards(rewards, #rewards at each step
                           gamma = 0.99 #discount for reward
                           ):
    """
    take a list of immediate rewards r(s,a) for the whole session 
    compute cumulative rewards R(s,a) (a.k.a. G(s,a) in Sutton '16)
    R_t = r_t + gamma*r_{t+1} + gamma^2*r_{t+2} + ...
    
    The simple way to compute cumulative rewards is to iterate from last to first time tick
    and compute R_t = r_t + gamma*R_{t+1} recurrently
    
    You must return an array/list of cumulative rewards with as many elements as in the initial rewards.
    """
    cum_rewards = len(rewards) * [0]
    cum_rewards[-1] = rewards[-1]
    for i in reversed(range(len(rewards)-1)):
        cum_rewards[i] = rewards[i] + gamma * cum_rewards[i+1]
      
    return cum_rewards

In [10]:
assert len(get_cumulative_rewards(range(100))) == 100
assert np.allclose(get_cumulative_rewards([0,0,1,0,0,1,0],gamma=0.9),[1.40049, 1.5561, 1.729, 0.81, 0.9, 1.0, 0.0])
assert np.allclose(get_cumulative_rewards([0,0,1,-2,3,-4,0],gamma=0.5), [0.0625, 0.125, 0.25, -1.5, 1.0, -4.0, 0.0])
assert np.allclose(get_cumulative_rewards([0,0,1,2,3,4,0],gamma=0), [0, 0, 1, 2, 3, 4, 0])
print("looks good!")

looks good!


In [11]:
def train_step(_states,_actions,_rewards):
    """given full session, trains agent with policy gradient"""
    _cumulative_rewards = get_cumulative_rewards(_rewards)
    update.run({states:_states,actions:_actions,cumulative_rewards:_cumulative_rewards})

### Playing the game

In [12]:
def generate_session(t_max=1000):
    """play env with REINFORCE agent and train at the session end"""
    
    #arrays to record session
    states,actions,rewards = [],[],[]
    
    s = env.reset()
    
    for t in range(t_max):
        
        #action probabilities array aka pi(a|s)
        action_probas = get_action_proba(s)
        
        a = np.random.choice([0,1],p=action_probas)
        
        new_s,r,done,info = env.step(a)
        
        #record session history to train later
        states.append(s)
        actions.append(a)
        rewards.append(r)
        
        s = new_s
        if done: break
            
    train_step(states,actions,rewards)
            
    return sum(rewards)
        

In [13]:
s = tf.InteractiveSession()
s.run(tf.global_variables_initializer())

for i in range(1000):
    
    rewards = [generate_session() for _ in range(1)] #generate new sessions
    
    print ("mean reward:%.3f"%(np.mean(rewards)))

    if np.mean(rewards) == 1000:
        print ("You Win!")
        break
        


mean reward:24.000
mean reward:40.000
mean reward:15.000
mean reward:51.000
mean reward:22.000
mean reward:12.000
mean reward:19.000
mean reward:34.000
mean reward:59.000
mean reward:12.000
mean reward:16.000
mean reward:28.000
mean reward:53.000
mean reward:64.000
mean reward:80.000
mean reward:25.000
mean reward:19.000
mean reward:38.000
mean reward:21.000
mean reward:25.000
mean reward:41.000
mean reward:31.000
mean reward:14.000
mean reward:14.000
mean reward:26.000
mean reward:50.000
mean reward:22.000
mean reward:10.000
mean reward:28.000
mean reward:45.000
mean reward:51.000
mean reward:22.000
mean reward:27.000
mean reward:31.000
mean reward:31.000
mean reward:40.000
mean reward:30.000
mean reward:28.000
mean reward:14.000
mean reward:39.000
mean reward:39.000
mean reward:17.000
mean reward:17.000
mean reward:48.000
mean reward:46.000
mean reward:63.000
mean reward:27.000
mean reward:17.000
mean reward:38.000
mean reward:38.000
mean reward:19.000
mean reward:27.000
mean reward:

mean reward:305.000
mean reward:322.000
mean reward:254.000
mean reward:185.000
mean reward:339.000
mean reward:369.000
mean reward:269.000
mean reward:265.000
mean reward:218.000
mean reward:262.000
mean reward:181.000
mean reward:270.000
mean reward:219.000
mean reward:245.000
mean reward:242.000
mean reward:206.000
mean reward:211.000
mean reward:178.000
mean reward:199.000
mean reward:250.000
mean reward:184.000
mean reward:194.000
mean reward:201.000
mean reward:181.000
mean reward:192.000
mean reward:235.000
mean reward:233.000
mean reward:233.000
mean reward:375.000
mean reward:195.000
mean reward:226.000
mean reward:260.000
mean reward:205.000
mean reward:201.000
mean reward:225.000
mean reward:285.000
mean reward:364.000
mean reward:208.000
mean reward:426.000
mean reward:401.000
mean reward:270.000
mean reward:320.000
mean reward:223.000
mean reward:314.000
mean reward:266.000
mean reward:273.000
mean reward:284.000
mean reward:213.000
mean reward:331.000
mean reward:317.000


### Results & video

In [32]:
#record sessions
import gym.wrappers
env = gym.wrappers.Monitor(gym.make("CartPole-v0"),directory="videos",force=True)
sessions = [generate_session() for _ in range(100)]
env.close()


In [33]:
#show video
from IPython.display import HTML
import os

video_names = list(filter(lambda s:s.endswith(".mp4"),os.listdir("./videos/")))

HTML("""
<video width="640" height="480" controls>
  <source src="{}" type="video/mp4">
</video>
""".format("./videos/"+video_names[-1])) #this may or may not be _last_ video. Try other indices

In [30]:
from submit import submit_cartpole
submit_cartpole(generate_session, '', '')

Submitted to Coursera platform. See results on assignment page!


In [31]:
# That's all, thank you for your attention!
# Not having enough? There's an actor-critic waiting for you in the honor section.
# But make sure you've seen the videos first.